## S&P 600 Galaxy

Use this utlity to update the returns and std_dev fields within investment-options.csv

Globals

In [1]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True
throttle_limit=100
wait_time=30

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [4]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

In [5]:
# Read in sp600.csv
sp600 = pd.read_csv('sp600.csv')
sp600.drop(columns=['SEC filings', 'CIK'], inplace=True)
sp600.rename(columns={'Company':'Description',
                      'GICS Sector':'Asset Class',
                      'GICS Sub-Industry': 'GICS Sub Industry'}, inplace=True)
sp600.set_index("Symbol", inplace=True)
sp600

,Description,Asset Class,GICS Sub Industry,Headquarters Location
Symbol,,,,
AAP,"Advance Auto Parts, Inc.",Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina"
AAT,American Assets Trust,Real Estate,Diversified REITs,"San Diego, California"
ABCB,Ameris Bancorp,Financials,Regional Banks,"Atlanta, Georgia"
ABG,Asbury Automotive Group,Consumer Discretionary,Automotive Retail,"Duluth, Georgia"
ABM,"ABM Industries, Inc.",Industrials,Environmental & Facilities Services,"New York City, New York"
...,...,...,...,...
XRX,Xerox,Information Technology,"Technology Hardware, Storage & Peripherals","Norwalk, Connecticut"
YELP,"Yelp, Inc.",Communication Services,Interactive Media & Services,"San Francisco, California"
YOU,"Clear Secure, Inc.",Information Technology,Application Software,"New York City, New York"


In [6]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [7]:
# Map sp600 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp600['Asset Class'] = sp600.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
sp600.index = sp600.index.str.replace('.', '-', regex=False)
sp600

,Description,Asset Class,GICS Sub Industry,Headquarters Location
Symbol,,,,
AAP,"Advance Auto Parts, Inc.",US Stocks:Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina"
AAT,American Assets Trust,US Stocks:Real Estate,Diversified REITs,"San Diego, California"
ABCB,Ameris Bancorp,US Stocks:Financials,Regional Banks,"Atlanta, Georgia"
ABG,Asbury Automotive Group,US Stocks:Consumer Discretionary,Automotive Retail,"Duluth, Georgia"
ABM,"ABM Industries, Inc.",US Stocks:Industrials,Environmental & Facilities Services,"New York City, New York"
...,...,...,...,...
XRX,Xerox,US Stocks:Technology,"Technology Hardware, Storage & Peripherals","Norwalk, Connecticut"
YELP,"Yelp, Inc.",US Stocks:Communication Services,Interactive Media & Services,"San Francisco, California"
YOU,"Clear Secure, Inc.",US Stocks:Technology,Application Software,"New York City, New York"


In [8]:
# Drop invalid symbols.
sp600.drop(['DRQ', 'SGH', 'UCBI'], inplace=True)


In [9]:
# Make symbols list.
symbols = list(sp600.index)
#symbols

In [10]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries, throttle_limit=throttle_limit, wait_time=wait_time)
bb.compile_timeseries(symbols)

AAP AAT ABCB ABG ABM ABR ACA ACAD ACIW ACLS ADEA ADMA ADNT ADUS AEIS AEO AESI AGO AGYS AHCO AHH AIN AIR AKR AL ALEX ALG ALGT ALK ALKS ALRM AMBC AMN AMPH AMR AMSF AMTM AMWD ANDE ANIP AORT AOSL APAM APLE APOG ARCB ARI ARLO AROC ARR ARWR ASIX ASO ASTE ASTH ATEN ATGE ATI AUB AVA AVAV AVNS AWI AWR AX AXL AZTA AZZ BANC BANF BANR BBWI BCC BCPC BDN BFH BFS BGC BGS BHE BHLB BJRI BKE BKU BL BLFS BLMN BMI BOH BOOT BOX BRC BRKL BSIG BTU BXMT CABO CAKE CAL CALM 
Throttle limit reached. Waiting for 0 seconds...
CALX CARG CARS CASH CATY CBRL CBU CCOI CCS CENT CENTA CENX CERT CEVA CFFN CHCO CHEF CLB CNK CNMD CNR CNS CNXN COHU COLL CON COOP CORT CPF CPK CPRX CRC CRGY CRI CRK CRSR CRVL CSGS CSR CSWI CTKB CTRE CTS CUBI CURB CVBF CVCO CVI CWEN CWEN-A CWK CWT CXM CXW DAN DCOM DEA DEI DFH DFIN DGII DIOD DLX DNOW DOCN DORM DRH DV DVAX DXC DXPE DY EAT ECG ECPG EFC EGBN EIG ELME EMBC ENOV ENR ENVA EPAC EPC EPRT ESE ETD ETSY EVTC EXPI EXTR EYE EZPW FBK FBNC FBP FBRT FCF FCPT 
Throttle limit reached. Waiting for

In [11]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df

,AAP,AAT,ABCB,ABG,ABM,ABR,ACA,ACAD,ACIW,ACLS,...,WT,WWW,XHR,XNCR,XPEL,XRX,YELP,YOU,ZD,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,142.24,30.21,29.21,68.12,27.82,5.35,27.59,16.65,27.14,17.79,...,5.93,28.31,14.35,35.79,NaN,14.49,34.57,NaN,59.27,22.48
2019-01-03,146.68,30.22,29.11,67.84,27.53,5.43,27.61,16.17,25.16,16.57,...,5.60,27.73,14.27,34.86,NaN,14.17,33.64,NaN,57.62,21.19
2019-01-04,143.05,30.46,30.25,68.47,28.47,5.58,28.32,17.57,26.17,17.61,...,5.75,28.32,14.79,36.58,NaN,14.72,34.56,NaN,60.02,22.03
2019-01-07,145.14,30.94,31.45,70.66,29.78,5.72,29.73,18.23,26.51,17.85,...,5.81,28.47,14.79,38.09,NaN,15.04,34.83,NaN,60.60,22.17
2019-01-08,144.14,31.58,31.37,69.83,31.11,5.81,29.50,18.54,27.39,17.78,...,5.96,28.89,15.29,38.01,NaN,15.53,35.51,NaN,60.88,22.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-10,36.58,20.81,56.86,247.92,51.25,12.32,78.55,17.19,51.69,59.97,...,8.39,14.02,13.06,13.63,30.08,6.26,35.25,25.30,39.70,34.34
2025-03-11,36.52,20.39,56.29,246.96,49.83,12.05,81.50,17.19,51.77,57.60,...,8.63,13.61,12.87,13.45,30.30,6.11,35.15,25.16,39.98,33.15
2025-03-12,37.58,20.26,56.48,243.03,45.51,12.26,80.11,16.77,51.61,59.42,...,8.60,13.43,12.78,13.13,30.57,6.05,34.88,25.02,40.05,32.83


In [12]:
# Sample symbol.
symbol = 'AAP'

In [13]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

np.float64(-51.16008994400705)

In [14]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [15]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

/tmp/ipykernel_4336/3923722811.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = df.pct_change()


np.float64(0.7785340303915241)

In [16]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

np.float64(0.6287868880495818)

In [17]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()

In [18]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

/tmp/ipykernel_4336/3878413049.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  monthly_returns = monthly.pct_change()


Date
2019-01-31     NaN
2019-02-28    0.02
2019-03-31    0.05
2019-04-30   -0.02
2019-05-31   -0.07
              ... 
2024-11-30    0.16
2024-12-31    0.14
2025-01-31    0.03
2025-02-28   -0.24
2025-03-31    0.02
Freq: ME, Name: AAP, Length: 75, dtype: float64

In [19]:
# Calculate 1 year, 3 year, and 5 year annualized standard deviation.
std_dev_1yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=1)
std_dev_3yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev_5yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=5)

In [20]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [21]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

# This is still slow (2.53 s).
for i, (index, row) in enumerate(sp600.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd_1yr = std_dev_1yr[symbol]*100
    sd_3yr = std_dev_3yr[symbol]*100
    sd_5yr = std_dev_5yr[symbol]*100

    out.append((
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}",'
        '"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
    ).format(
        symbol, description, asset_class, ret_1mo, ret_3mo, ret_1yr, ret_3yr,
        ret_5yr, _vola, _ds_vola, sd_1yr, sd_3yr, sd_5yr
    ))

In [22]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')